# 뉴럴 네트워크 학습 알고리즘 구현

## Import modules

In [1]:
import tensorflow as tf

## 하이퍼파라미터 정의

In [2]:
EPOCHS = 10

## 네트워크 구조 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.dense3 = tf.keras.layers.Dense(128, activation='relu')
        self.dense4 = tf.keras.layers.Dense(256, activation='relu')
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')       
    
    def call(self, x, training=None, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.dense5(x)

## 학습 함수 구현

In [18]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)  # BatchSize x 10(Classes)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

## 테스트 함수 구현

In [19]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)  # BatchSize x 10(Classes)
    loss = loss_object(labels, predictions)
    
    test_loss(loss)
    test_accuracy(labels, predictions)

## 데이터 불러오기

In [20]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 모델 생성

In [21]:
model = MyModel()

## 손실함수 및 최적화 알고리즘 정의

In [22]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 성능 지표 정의

In [23]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 구현

In [25]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    for images, labels in test_ds:
        test_step(model, images, labels, loss_object, test_loss, test_accuracy)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch +1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.21706123650074005, Accuracy: 93.350830078125, Test Loss: 0.17650191485881805, Test Accuracy: 94.70000457763672
Epoch 2, Loss: 0.1102803647518158, Accuracy: 96.62166595458984, Test Loss: 0.14457902312278748, Test Accuracy: 95.52000427246094
Epoch 3, Loss: 0.08990293741226196, Accuracy: 97.22000122070312, Test Loss: 0.1462688148021698, Test Accuracy: 95.81999969482422
Epoch 4, Loss: 0.07684317976236343, Accuracy: 97.61000061035156, Test Loss: 0.12955234944820404, Test Accuracy: 96.41999816894531
Epoch 5, Loss: 0.06661977618932724, Accuracy: 97.86499786376953, Test Loss: 0.14261017739772797, Test Accuracy: 96.31999969482422
Epoch 6, Loss: 0.06150249391794205, Accuracy: 98.05333709716797, Test Loss: 0.1897428035736084, Test Accuracy: 95.20000457763672
Epoch 7, Loss: 0.05350516363978386, Accuracy: 98.22999572753906, Test Loss: 0.13986477255821228, Test Accuracy: 96.59000396728516
Epoch 8, Loss: 0.05047263950109482, Accuracy: 98.29000091552734, Test Loss: 0.1346091330051422,